<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>
    
<h3>Curso Ingeniería de Características</h3>

<h2>Descargando datos de XML</h2>

<h4> Alumna: Diana Laura Ballesteros Valenzuela</h4>

<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>
    
</center>

Importando librerias necesarias

In [1]:
import re
import pandas as pd
import requests
import xml.dom.minidom
import numpy as np
import xml.etree.cElementTree as et
from inspect import getmembers , isclass , isfunction

Enlace al XML del que queremos obtener la información 

In [2]:
XML= "https://raw.githubusercontent.com/mcd-unison/ing-caract/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"

In [3]:
response = requests.get(XML)

with open('poetas.xml', 'wb') as file:
    file.write(response.content)
    
dom = xml.dom.minidom.parse("./poetas.xml") 
pretty = dom.toprettyxml()

In [4]:
tree=et.parse('./poetas.xml') #Leemos archivo
root=tree.getroot()

In [5]:
lista_autores = []
text_list = []

for autor in root.findall('{http://www.mediawiki.org/xml/export-0.10/}page'):
    title =  autor.find('{http://www.mediawiki.org/xml/export-0.10/}title')
    revision =  autor.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
    texto = revision.find('{http://www.mediawiki.org/xml/export-0.10/}text')
    
    lista_autores.append(title.text)
    text_list.append(texto.text)

In [6]:
df_autores = pd.DataFrame(lista_autores) #Dataframe para autores
df_autores.columns = ['Autor']

df_text = pd.DataFrame(text_list) #dataframe para texto
df_text.columns = ['Texto']

df = pd.concat([df_autores, df_text], axis=1, join='inner')

In [7]:
novela = []
for text in df['Texto']: # Filtrar cada obra
    novela_found = re.findall("(==\s*Novela\w*\s*==)(.*?)(\n==)", text, flags = re.DOTALL)
    novela.append(novela_found)

In [8]:
dfNovela = pd.DataFrame(novela)
dfNovela.columns = ['Novela']
dfNovela 

df = pd.concat([df, dfNovela], axis=1, join='inner') 
df.to_parquet('poetas.parquet')  #dataframe con obra de cada autor

Visualizando dataframe final

In [9]:
pd.read_parquet('poetas.parquet', engine='pyarrow')

,Autor,Texto,Novela
0,Julia Morilla de Campbell,'''Julia Morilla de Campbell''' ([[Rosario (Ar...,None
1,Luis Negreti,{{Ficha de escritor\n|Imagen = NE...,None
2,Poldy Bird,{{Ficha de persona\n| padres = Enrique Bird Mo...,None
3,Ana María Shua,{{Ficha de persona\n|imagen=\n|nombre de nacim...,"[== Novelas ==, =\n* 1980 - ''Soy paciente'', ..."
4,León Benarós,{{Ficha de persona\n|nombre = León B...,None
...,...,...,...
634,Humberto Tejera,{{Ficha de persona\n| nombre = Humberto Tejera...,None
635,Mario Molina Cruz,{{Ficha de escritor\n|nombre= Mario Molina Cru...,"[== Novela ==, =\n* ''Xtille Zikw Belé, lhén b..."
636,Luis Ignacio Helguera,{{Ficha de persona\n| nombre = Luis Ignacio He...,None
637,Daniel Olivares Viniegra,{{Promocional|8|julio}}\n\n'''Daniel Olivares ...,None
